# Identifying the common and the rare in Census data

We can use density estimation to identify a "prototypical American" in Census data. Similarly, we can identify some highly unusual demographics. We'll use the Census data that the SHAP package makes available (exact provenance unknown):

In [1]:
import pandas as pd
import pydens

df = pydens.load_SHAP_census_data()

We'll integer-code the categorical variables because CADE currently requires numeric inputs. Then we train CADE on the census data and sort the original data according to the fitted density:

In [2]:
categorical_cols = [col for col in df.columns if df[col].dtype.name == 'category']
num_data = df.copy()
for col in categorical_cols:
    num_data[col] = num_data[col].cat.codes

classifier = pydens.classifiers.lightgbm.Lgbm(categorical_features=categorical_cols)
cade = pydens.cade.Cade(
    initial_density=pydens.models.JointDensity(),
    classifier=classifier
)
cade.train(num_data, diagnostics=True)
cade.diagnostics['auc']

0.9465986331882514

In [3]:
df['density'] = cade.density(num_data)
ddf = df.copy().drop_duplicates()
ddf.sort_values('density', inplace=True)

## A typical American

The most common adult American demographic (in terms of the sample participants and the features that the census collects) is a mid-30s married white male who leverages a high school diploma to earn less than 50k while working 40 hours per week for a private employer in 'craft-repair' and accrues no large capital gains or losses.

In [4]:
ddf.tail()

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Income,density
5086,33.0,Private,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,<=50K,0.000103
2675,34.0,Private,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,<=50K,0.000104
6266,31.0,Private,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,<=50K,0.000104
3277,35.0,Private,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,<=50K,0.000105
65,36.0,Private,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,<=50K,0.000107


## Examples of unusual demographics

The most unusual demographics are those at points of lowest density. Here are a few examples:

In [5]:
df.head()

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Income,density
0,39.0,State-gov,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,<=50K,5.358149e-10
1,50.0,Self-emp-not-inc,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,<=50K,3.443545e-09
2,38.0,Private,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,<=50K,4.529538e-06
3,53.0,Private,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,<=50K,1.909915e-07
5,37.0,Private,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,<=50K,7.255463e-07


In [6]:
# import shmistogram as shmist
# shm = shmist.Shmistogram(df.Age.values)
# shm.plot()